# HOJA DE TRABAJO 2
## MARIA FERNANDA 14198

Primero importamos las funciones del archivo metodos_redes.py que contienen los algoritmos de back propagation, feed forward, costo, calculo del jacobiano.

In [3]:
from metodos_redes import funcion_costo, formar_matrices, jacobiano, feed_forward
from mnist_reader import load_mnist
import numpy as np

### Set de datos

Se utilizará para este analisis el set de datos fashion-mnist que contiene una variedad de imagenes a blanco y negro representando 10 diferentes categorías de ropa. Como set de entrenamiento se utilizara el conjunto de imagenes train que consiste de 60000 imagenes y para las pruebas se utilizara el conjunto test que contiene 10000 imagenes.

In [ ]:
# Leer datos de csv
train_data = np.genfromtxt('fashion-mnist_train.csv', delimiter=',', skip_header=1)

X_entrenamiento = np.delete(train_data, 0, 1)
y_entrenamiento = [row[0] for row in train_data]

In [6]:
test_data = np.genfromtxt('fashion-mnist_test.csv', delimiter=',', skip_header=1)

X_prueba = np.delete(test_data, 0, 1)
y_prueba = [row[0] for row in test_data]

### Entrenando el modelo

Para entrenar el modelo se definarán primero las capas. Nuestro modelo tendrá 3 capas diferentes. La capa de entrada está compuesta de una neurona por pixel de las imagenes. La capa oculta esta compuesta por 100 neuronas. La capa de salida esta compuesta de 10 neuronas cada una correspondiente a uno de los tipos de ropa.

Antes de realizar el entrenamiento normalizamos los datos de las imagenes para llevarlos de un rango de 0 a 1. Las clases en Y deben de ser transformadas de un rango de un entero a su representacion en un arreglo de diez posiciones, donde la primera clase se ve de la forma [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
import numpy as np
import time
import pickle
import math
import scipy
from scipy.optimize import minimize
from functools import reduce

sigmoid = lambda x: 1.0 / (1.0 + np.exp(-x))

flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

X_normalizado = X_entrenamiento / 1000.0
m, n = X_normalizado.shape

Y_modificado = y_entrenamiento.reshape(m, 1)
Y_modificado = (Y_modificado == np.array(range(10))).astype(int)

layers = np.array([
    n, # Se utilizaran el numero de pixeles como capa de entrada
    100, # Se utilizaran 100 neuronas en la capa oculta
    10 # Estas son las clases a identificar
])

theta_shapes = np.hstack((
    layers[1:].reshape(len(layers) - 1, 1),
    (layers[:-1] + 1).reshape(len(layers) - 1, 1)
))

flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape)
    for theta_shape in theta_shapes
])

Realizamos el proceso de entrenamiento. Este proceso tardo al rededor de 1 hora en completarse.

In [ ]:
print("ENTRENANDO!")

result = minimize(
    fun = funcion_costo,
    x0 = flat_thetas,
    args = (theta_shapes, X_normalizado, Y_modificado),
    method = 'L-BFGS-B',
    jac = jacobiano,
    options = {'disp': True, 'maxiter': 1000} # Se establecen 1000 iteraciones como max
)

outfile = open('modelo', 'wb')
pickle.dump(result.x, outfile)
outfile.close()

print("FINALIZADO!")

### Set de prueba

Para probar nuestro modelo haremos el mismo procesamiento de datos que al entrenarlo. Luego definimos la misma estructura de la red neuronal y procedemos a realizar las predicciones.

In [7]:
from scipy.optimize import minimize
import numpy as np
import pickle
from functools import reduce

flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

X_normalizado = X_prueba / 1000.0
m, n = X_normalizado.shape

layers = np.array([
    n,
    100,
    10
])

theta_shapes = np.hstack((
    layers[1:].reshape(len(layers) - 1, 1),
    (layers[:-1] + 1).reshape(len(layers) - 1, 1)
))

openfile = open("modelo", "rb")
thetas_trained = pickle.load(openfile)

thetas = formar_matrices(thetas_trained, theta_shapes)

predicciones = feed_forward(thetas, X_normalizado)[-1]

### Resultados


Se comparan las predicciones con las clases reales proveidas por el set de datos.

In [8]:
matches = 0

for i in range(m):
    clase = np.where((predicciones[i] == np.amax(predicciones[i])))[0]

    if(clase == y_prueba[i]):
        matches += 1
    

precision = (matches/m)*100

print('Matches: ' + str(matches))
print('Precision: ' + str(precision) + '%')

Matches: 8604
Precision: 86.04%


Se puede observar que la precisión del algoritmo fue bastante buena con un 86.04%. Se probaron +-10 capas ocultas y esta configuracion dió el mejor resultado.